In [1]:
from src.rapidata_client.rapidata_client import RapidataClient
from src.rapidata_client.order.feature_flags.feature_flags import FeatureFlags

import dotenv
import os
dotenv.load_dotenv()

CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
ENDPOINT = os.getenv("ENDPOINT")

if not CLIENT_ID:
    raise Exception("CLIENT_ID not found in environment variables")

if not CLIENT_SECRET:
    raise Exception("CLIENT_SECRET not found in environment variables")

if not ENDPOINT:
    raise Exception("ENDPOINT not found in environment variables")

## Basic Classify Order

In [2]:
### The rapidata client is the entry point to the Rapidata API
rapi = RapidataClient(
    client_id=CLIENT_ID, client_secret=CLIENT_SECRET, endpoint=ENDPOINT
)

In [3]:
from src.rapidata_client.order.workflow.classify_workflow import ClassifyWorkflow
from src.rapidata_client.order.referee.naive_referee import NaiveReferee

# Configure order
order = (
    rapi.new_order(
        name="Example Classify Order",
    )
    .workflow(
        ClassifyWorkflow(
            question="Who should be president?",
            categories=["Kamala Harris", "Donald Trump"],
        )
        .referee(NaiveReferee(required_guesses=15))
        .feature_flags(FeatureFlags().alert_on_fast_response(3))
    )
    .create()
)

# Add data
order.dataset.add_images_from_paths(["examples/data/kamala_trump.jpg"])

# Let's go!
order.submit()
# order.approve() admin only: if it doesn't auto approve and you want to manually approve

Exception: Error: 400 - {"type":"https://tools.ietf.org/html/rfc9110#section-15.5.1","title":"One or more validation errors occurred.","status":400,"errors":{"model":["The model field is required."],"$.workflowConfig.blueprint":["Read unrecognized type discriminator id 'AttachCategoryRapidBlueprint'. Path: $.workflowConfig.blueprint | LineNumber: 0 | BytePositionInLine: 371."]},"traceId":"00-ae0fee98f48d2755efa675cea794095a-67202ccef832b5c1-01"}

## Compare Order

In [ ]:
from src.rapidata_client.order.workflow.compare_workflow import CompareWorkflow


order = rapi.new_order(
    name="Example Compare Order",
).workflow(
    CompareWorkflow(
        criteria="Who should be president?",
    )
    .matches_until_completed(5)
    .match_size(2)
).create()

order.dataset.add_images_from_paths(["examples/data/kamala.jpg", "examples/data/trump.jpg"])

order.submit()
# order.approve()

## Free Text Input

In [ ]:
from src.rapidata_client.order.country_codes.country_codes import CountryCodes
from src.rapidata_client.order.workflow.free_text_workflow import FreeTextWorkflow


order = rapi.new_order(
        name="Example Video Free Text Order",
    ).workflow(
        FreeTextWorkflow(
            question="Describe the movement in this video!",
        ).feature_flags(
            FeatureFlags().free_text_minimum_characters(15).alert_on_fast_response(5)
        ).target_country_codes(CountryCodes.ENGLISH_SPEAKING)
    ).create()

order.dataset.add_videos_from_paths([""]) # TODO: insert video path
order.submit()

In [ ]:
order.approve()